In [1]:
import math
import json
import numpy as np
import pandas as pd
import networkx as nx
#import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from IPython.display import Image
%matplotlib inline
import os
import conda

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

from mpl_toolkits.basemap import Basemap

In [27]:
cities = pd.read_csv('data/Top50/top50cities.csv',delimiter=',')

In [28]:
#import nodes to get id from nodes/hosts
hosts = pd.DataFrame() #creates a new dataframe that's empty
for idx,city in cities.iterrows():#.loc[cities['2017']>50000].iterrows():
    for p in range(1,14):
        if (os.path.exists('data/Top50/hosts/'+city['name']+str(p)+'.csv')):
            df = pd.read_csv('data/Top50/hosts/'+city['name']+str(p)+'.csv',index_col=False)
            hosts = pd.concat([hosts,df]).reset_index(drop=True)
print(len(hosts))
hosts=hosts.drop_duplicates()
print(len(hosts))


10586
10582


In [30]:
rev = pd.DataFrame() #creates a new dataframe that's empty
for idx,city in cities.iterrows():#.loc[cities['2017']>50000].iterrows():
    if os.path.exists('data/Top50/reviews/'+city['name']+'Rev.csv'):
        df = pd.read_csv('data/Top50/reviews/'+city['name']+'Rev.csv',index_col=False)
        rev = pd.concat([rev,df]).reset_index(drop=True)
print(len(rev))
rev=rev.drop_duplicates()
print(len(rev))


96090
96085


In [5]:
!pip install geopy
from geopy import geocoders  


In [50]:
cities_not_found = []
def getCityCoords(name):
    # if geopy does not work -> http://www.geonames.org/login and log in with cred:
    # username: kacper
    # pass : zaq12345
    gn = geocoders.GeoNames(username='kacper')
    try:
        loc = gn.geocode(name, timeout=10)
        
        if loc is None:
            try: 
                loc =gn.geocode(name.split(',')[0],timeout=10)
                if loc is None:
                    raise TypeError
            except Exception as inst:
                print('This city was not found:',name)
                return None
    except Exception as inst:
        print('This city was not found:',name)
        cities_not_found.append(name)
        return None
    #print(len(loc))
   # print(loc[0].raw)
    return loc

In [ ]:
#loc
#loc = getCityCoords(name)
#rev['lat']='NaN'
#rev['lng']='NaN'
#allCities=rev.fromPublicAddressDescription.unique()
#for city in allCities.tolist():
#    loc2=getCityCoords(city)
#    if loc2:
#        rev.loc[rev.fromPublicAddressDescription==city]['lat']=loc2.raw['lat']
#        rev.loc[rev.fromPublicAddressDescription==city]['lng']=loc2.raw['lng']






for idx,row in rev.iterrows():
    if row['lat']=='NaN':
        print(idx)
        #print(row['fromPublicAddressDescription'])
        loc2=getCityCoords(row['fromPublicAddressDescription'])
        if loc2:
            #print(loc.raw['lat'],loc.raw['lng'])
            rev['lat'][idx]=loc2.raw['lat']
            rev['lng'][idx]=loc2.raw['lng']

        #rev.loc[idx]['lng']

182
This city was not found: Bilhorod-Dnistrovs'kyi, Odessa Oblast, Ukraine
197
This city was not found: Bilhorod-Dnistrovs'kyi, Odessa Oblast, Ukraine
207
This city was not found: Yenakijeve, Donetsk Oblast, Ukraine
580
This city was not found: guayaqil, Ecuador
977
This city was not found: La boulladisse, Provance Alpes, France
1173
This city was not found: Pervomais'k, Mykolaivs'ka oblast, Ukraine
1351
This city was not found: khajurajo, Madhya Pradesh, India
1645
This city was not found: Volgogard, Volgograd Oblast, Russian Federation
2246
This city was not found: i was born in, Italy
2253
This city was not found: Plyos, Ivanovo Oblast. Привллжский район,, Russian Federation
2307


/Users/Eike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/Eike/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


2308
2309
2310
2311
2312
2313
2314
2315
2316
2317
2318
2319
2466
This city was not found: ChIJq5EYs65unUcRC-cFw9pQGh4
2514
This city was not found: Guishan dist, Taoyuan, Taiwan, Province of China
2622
This city was not found: Khmel'nyts'kyi, Khmel'nyts'ka oblast, Ukraine
2833
This city was not found: Olmalik, Uzbekistan
2983
This city was not found: Guishan dist, Taoyuan, Taiwan, Province of China
3345
This city was not found: Clovis (Yosemite), CA, USA
3425
This city was not found: Rēzekne Municipality, Latvia
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710


In [70]:
rev
#rev.to_csv('data/reviews/AllreviewsWithCordinates', index=False)

,id,text,createdDate,experience,relationshipType,isPostTrip,to,from,fromPublicName,fromPublicAddressId,fromPublicAddressDescription,fromAvatarUrl,fromStatus,fromIsVerified,fromIsDeleted,fromBlockedBy,inverseReference,response,lat,lng
0,27077307,Amazing chef and humble human being. Willing t...,2018-10-17T22:09:22Z,positive,host,True,2009059021,2001999679,Dhanush Anand,109610804.0,"Setiabudi, Jakarta, Indonesia",https://s3.amazonaws.com/ht-images.couchsurfin...,yes,True,False,NaN,NaN,NaN,-6.2182,106.8306
1,27152728,Amazing boy with amazing family . He met me up...,2018-10-28T19:16:31Z,positive,surf,True,2009059021,2007724821,Bakhar Sardzhaeva,271153483.0,Dubai,https://s3.amazonaws.com/ht-images.couchsurfin...,no,False,False,NaN,NaN,NaN,25.0657,55.17128
2,26922175,￼\n\n \n\nWe use cookies to enhance your exper...,2018-09-27T14:41:06Z,positive,surf,True,2008295707,2008797517,Sun-hyung Kim,312790604.0,"Seoul, Seoul, Korea (Republic of)",https://s3.amazonaws.com/ht-images.couchsurfin...,maybe,False,False,NaN,NaN,NaN,37.566,126.9784
3,26866500,"I was ill, when I come to Alexey's place, so I...",2018-09-20T17:32:11Z,positive,surf,True,2008295707,2003041472,Michelle Kuglerová,123277124.0,"Jihlava, Vysocina Region, Czechia",https://s3.amazonaws.com/ht-images.couchsurfin...,maybe,False,False,NaN,NaN,NaN,49.3961,15.59124
4,26818885,Didn't end up staying at his house because of....,2018-09-15T05:17:31Z,positive,surf,True,2008295707,4294146,Yvette Shaw,6999267.0,"Paihia, Northland, New Zealand",https://s3.amazonaws.com/images.couchsurfing.u...,no,False,False,NaN,NaN,NaN,-35.28067,174.09103
5,26779351,С Алексеем было очень приятно и комфортно пров...,2018-09-10T12:56:37Z,positive,surf,True,2008295707,2008843836,Angelika Krasnozhonova,314663738.0,"Kirov Oblast, Russian Federation",https://s3.amazonaws.com/ht-images.couchsurfin...,hang,True,False,NaN,NaN,NaN,58.59665,49.66007
6,26176442,I had a great experience in Russia with Alexei...,2018-07-10T13:57:26Z,positive,surf,True,2008295707,2007359140,Diego Gabriel Cortez Fernandez,250795609.0,"Hangzhou, xiasha, Zhejiang, China",https://s3.amazonaws.com/ht-images.couchsurfin...,no,False,False,NaN,NaN,NaN,30.29365,120.16142
7,26164016,I bet Alexey is definitely a good person. He l...,2018-07-09T09:52:29Z,positive,surf,True,2008295707,2007634207,Heejeong Yang,265596282.0,"Seoul, Seoul, Korea, Republic of",https://s3.amazonaws.com/ht-images.couchsurfin...,no,False,False,NaN,NaN,NaN,37.566,126.9784
8,26148751,My experience with Alexey was awesome. He is ...,2018-07-07T16:05:53Z,positive,surf,True,2008295707,2005150941,Siboney Godoy,176314139.0,"MD, USA",https://s3.amazonaws.com/ht-images.couchsurfin...,hang,False,False,NaN,NaN,NaN,39.28808,-76.61076
9,26119628,Alexey is a really amazing host!! At the first...,2018-07-04T09:15:01Z,positive,surf,True,2008295707,1003812034,Emma Jimenez,10585071.0,"Pereira, Risaralda, Colombia",https://s3.amazonaws.com/images.couchsurfing.u...,maybe,False,False,NaN,NaN,NaN,4.81333,-75.69611


In [ ]:
G = nx.MultiDiGraph()
links = []
user_id_unique=set(hosts['id'].tolist())
G.add_nodes_from([(rows['name'],{'lat':rows['lat'],'lng':rows['lng']}) for idx,rows in cities.iterrows()])
G.add_nodes_from([(rows['fromPublicAddressDescription'],{'lat':rows['lat'],'lng':rows['lng']}) for idx,rows in rev.iterrows() if rows['lat'] !='NaN' rows['lng']!='NaN')
for idx,r in rev.iterrows():
    #if r['from'] in hosts['id'].tolist() and r['to'] in hosts['id'].tolist():
    if r['from'] in G.nodes().tolist()
        if r['relationshipType']=='surf':
            G.add_edge(hosts.loc[hosts['id']==r['from']]['city'].item(),hosts.loc[hosts['id']==r['to']]['city'].item())
        if r['relationshipType']=='host':
            G.add_edge(hosts.loc[hosts['id']==r['to']]['city'].item(),hosts.loc[hosts['id']==r['from']]['city'].item())
#G.add_edges_from([(rows['to'],rows['from'])for index,rows in rev[['to','from','relationshipType']].iterrows() if rows['relationshipType']=='host' and G.has_node(rows['to']) and G.has_node(rows['from'])])
#G.add_edges_from([(rows['from'],rows['to'])for index,rows in rev[['to','from','relationshipType']].iterrows() if rows['relationshipType']=='surf' and G.has_node(rows['to']) and G.has_node(rows['from'])])



In [ ]:
plt.figure(figsize = (10,9))
m = Basemap(projection='merc',llcrnrlon=-30,llcrnrlat=25,urcrnrlon=50,
urcrnrlat=70, lat_ts=0, resolution='l',suppress_ticks=True)
mx, my = m(cities['lng'].values, cities['lat'].values)
pos = {}
for count, elem in enumerate (cities['name']):
     pos[elem] = (mx[count], my[count])

nx.draw_networkx_nodes(G, pos = pos, node_list =G.nodes() ,node_color = 'firebrick', alpha = 0.8, node_size = [v[1] * 3 for v in G.degree()])
nx.draw_networkx_edges(G, pos = pos, edge_color='cornflowerblue', alpha=0.2, arrows = False)
#nx.draw_networkx_edges(G, pos = pos, edge_color='g', alpha=0.2, arrows = False)
m.drawcountries(linewidth = 3)
#m.drawstates(linewidth = 0.2)
m.drawcoastlines(linewidth=3)
m.bluemarble()
plt.tight_layout()
plt.savefig("map.png", format = "png", dpi = 300)

plt.show()